Welcome to the example code to use PIXIS camera with the Monochromator. 
We first being with importing relevant python pachanges and the API for the PIXIS Camera: 

In [1]:
import codecs
import time
import serial
import pandas as pd
import numpy as np
import pyvisa as visa
import datetime
import os
import sys
import monochromatorapi as mcapi # Import monochromator api
import clr # Import the .NET class library
import sys # Import python sys module
import os # Import os module
from System.IO import * # Import System.IO for saving and opening files
# Import C compatible List and String
from System import String
from System.Collections.Generic import List
# Add needed dll references
sys.path.append(os.environ['LIGHTFIELD_ROOT'])
sys.path.append(os.environ['LIGHTFIELD_ROOT']+"\\AddInViews")
clr.AddReference('PrincetonInstruments.LightFieldViewV5')
clr.AddReference('PrincetonInstruments.LightField.AutomationV5')
clr.AddReference('PrincetonInstruments.LightFieldAddInSupportServices')
# PI imports
from PrincetonInstruments.LightField.Automation import Automation
from PrincetonInstruments.LightField.AddIns import ExperimentSettings
from PrincetonInstruments.LightField.AddIns import CameraSettings
from PrincetonInstruments.LightField.AddIns import DeviceType
from PrincetonInstruments.LightField.AddIns import SensorTemperatureStatus
from PrincetonInstruments.LightField.AddIns import TriggerResponse
import PrincetonInstruments.LightField.AddIns as AddIns
# Create the LightField Application (true for visible)
# The 2nd parameter forces LF to load with no experiment 
 
def pixis_load_experiment(filename='DemoExp'):
    """
    Import premade settings file in LightField to send commands through api instead.
        Inputs:
            :filename(string): Experiment setting file premade in Lightfield
        Returns:
            ::Warning if Dummy Camera settings are loaded
            ::Connection message
            ::Error if not connected
    """
    if filename=='DemoExp':
        print("Warning: Loading Dummy Camera. Please select input filename to connect to the real PIXIS")
    auto = Automation(True, List[String]()) #send commands from code instead of LF GUI
    experiment = auto.LightFieldApplication.Experiment #settings from LF file user premade
    connected=experiment.Load(filename) #load settings for LF
    if connected == False: 
        print("Warning: Could not connect to the camera. Check Light Field Window or Experiment file name.")
    elif connected == True: 
        print("Connection successful")
    return connected, auto, experiment

def pixis_set_value(setting, value):
    """
    Check for existing experiment before setting new gain, adc rate, or adc quality.
        Inputs:
            :setting(string):
            :value():
    """    
    if experiment.Exists(setting):
        experiment.SetValue(setting, value)

def pixis_get_value(setting):
    """
    Check for settings for gain, adc rate, adc quality, return values
        Inputs:
            :setting(string):
    """   
    if experiment.Exists(setting):
        return experiment.GetValue(setting)

def pixis_device_found():
    """
    Find connected device an inform user if device not detected
        Returns:
            ::Error message
    """
    for device in experiment.ExperimentDevices:
        if (device.Type == DeviceType.Camera):
            return True
    print("Camera not found. Please add a camera and try again.") # If connected device is not a camera inform the user
    return False  

def pixis_take_bias_frames(n_bias):
    """
    Loop to capture bias images with PIXIS.
        Inputs:
            :n_bias(float): number of bias frames
        Returns:
            ::message that loop is beginning
    """
    print(f"Taking {n_bias} frames")
    bias_exposure_time=0 #exposure time set to 0 for bias frames 
            #may need to set the shutter function of PIXIS to not send trigger to ensure shutter does not open. 
            #Right now the assumption is that the shutter does not open when a 0 second exopsure is given. 
            #Set exposure time
    count =1
    while count<=n_bias: 
        count+=1
        pixis_set_value(CameraSettings.ShutterTimingExposureTime, bias_exposure_time)
        pixis_set_value(CameraSettings.ShutterTimingMode,3)
        if (pixis_device_found()==True): 
                # Acquire image
            waitUntil_ready(5)
            experiment.Acquire()
            waitUntil_ready(5)
            pixis_set_value(CameraSettings.ShutterTimingExposureTime, bias_exposure_time)
    pixis_set_value(CameraSettings.ShutterTimingMode,2)

def pixis_take_exposure(exposuretime):
    """
    Opens shutter, triggers LightField Aquire command, closes shutter.
        Inputs:
            :exposuretime(float): exposure time (seconds)
        Returns:
            ::Exposure time value, and completion
    """ 
    if (pixis_device_found()==True): 
            pixis_set_value(CameraSettings.ShutterTimingMode,2)
            pixis_set_value(CameraSettings.ShutterTimingExposureTime, exposuretime)
            print(f"exposure time set to {exposuretime}")
            waitUntil_ready()
            experiment.Acquire()
            waitUntil_ready()
            print("Exposure complete")

def pixis_take_dark_frames(n_dark,exposure_time):
    """
    Aquire images with shutter closed.
        Inputs:
            :n_dark(float): number of dark frames
            :exposure_time(float): time shutter is open
        Returns:
            ::Message that process is starting
    """
    print(f"Taking {n_dark} dark frames")
    count =1 
    while count<=n_dark: 
        count+=1
        if (pixis_device_found()==True): 
            waitUntil_ready()
            pixis_set_value(CameraSettings.ShutterTimingMode,3)
            pixis_set_value(CameraSettings.ShutterTimingExposureTime, exposure_time)
            waitUntil_ready()
            experiment.Acquire()
            waitUntil_ready()
    pixis_set_value(CameraSettings.ShutterTimingMode,2)

def waitUntil_ready(delay=10): #default value is 5 seconds
    """
    Delay for each command.
        Inputs:
            :delay(float): Time in seconds
    """ 
    ":delay: delay in seconds"
    condition=experiment.IsReadyToRun
    wU = True
    while wU == True:
        if condition: #checks the condition
            wU = False
        time.sleep(delay) #waits 60s for preformance

def pixis_set_folder(exp_folder,parent_diretory):
    """
    Create new folder to store the experiment files and subfiles.
        Inputs:
            :exp_folder(string): folder name
            :parent_directory(string): location of folder
        Returns:
            ::folder and directory creation messages
            ::error message, if error code is 17, updates folder path and directory
    """
    folder_location=parent_diretory+'\\'+exp_folder
    import os # importing os module   
    path = folder_location # path
    # Create the directory in '/home / User / Documents' 
    try: 
        os.mkdir(path)
        customdirectory=path
        print(f"Created empty folder {customdirectory} for storing experiment files")
        pixis_set_value(ExperimentSettings.FileNameGenerationDirectory,customdirectory)
        pixis_set_value(ExperimentSettings.OnlineExportOutputOptionsCustomDirectory,customdirectory)
        print(f"Custom directory set to {customdirectory} ")
    except OSError as error: 
        out=error
        if out.errno==17: 
            customdirectory=path
            print(error)
            print(f"Path updated to existing {customdirectory}. Check that the folder is empty before proceeding")
            pixis_set_value(ExperimentSettings.FileNameGenerationDirectory,customdirectory)
            pixis_set_value(ExperimentSettings.OnlineExportOutputOptionsCustomDirectory,customdirectory)
            print(f"Custom directory set to {customdirectory} ")
            return(customdirectory)

def pixis_save_file(filename):
    """
    Set base file name, set increment, date, time.
        Inputs:
            :exp_folder(string): folder name
            :parent_directory(string): location of folder
        Returns:
            ::folder and directory creation messages
            ::error message, if error code is 17, updates folder path and directory
    """        
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationBaseFileName,
        Path.GetFileName(filename))
    # Option to Increment, set to false will not increment
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachIncrement,
        True)
    # Option to add date
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachDate,
        True)
    # Option to add time
    experiment.SetValue(
        ExperimentSettings.FileNameGenerationAttachTime,
        True)

def pixis_set_temperature(temperature):
    """
    Set temperature if Ready condition is met and not aquiring data.
        Inputs:
            :temperature(float): value in Celsius
        Returns:
            ::Temperature change message and set message
    """   
    if (experiment.IsReadyToRun & experiment.IsRunning==False):
        experiment.SetValue(CameraSettings.SensorTemperatureSetPoint,temperature)    
        time.sleep(5)
        while pixis_get_temperature_status()!=2:
            time.sleep(4)
            ct=pixis_get_current_temperature()
            print(f"Detector temperature is being changed to {temperature}. Current temperature is {ct}")
        print(pixis_get_temperature_status())
        print(f"Detctor temperature now set to {pixis_get_current_temperature()}")

def pixis_get_current_temperature():
    """
    Present temperature value of PIXIS for user to read.
        Returns:
            ::temperature in Celsius
    """
    current_temp=experiment.GetValue(CameraSettings.SensorTemperatureReading)
    print(f"Current Temperature:{current_temp}")
    return current_temp
    
def pixis_get_current_setpoint():
    """
    Present temperature setpoint for user to read.
        Returns:
            ::Set point in Celsius
    """
    print(String.Format(
        "{0} {1}", "Current Temperature Set Point:",
        experiment.GetValue(CameraSettings.SensorTemperatureSetPoint)))        

def pixis_get_temperature_status():
    """
    Read Status of Lock and Unlock feature based on temperature.
        Returns:
            ::Status and if Locked or not
    """    
    current = experiment.GetValue(CameraSettings.SensorTemperatureStatus)
    print(String.Format(
        "{0} {1}", "Current Status:",
        "UnLocked" if current == SensorTemperatureStatus.Unlocked 
        else "Locked"))
    return current


<ipython-input-1-cbf39a8542e9>:17: DeprecationWarning: The module was found, but not in a referenced namespace.
Implicit loading is deprecated. Please use clr.AddReference('System.Collections').
  from System.Collections.Generic import List


In [3]:
MC_dispersion_scale = 1.24 #nm/mm for 1200 g/mm grating from Model 207V Manual
MC_resolution = 0.04 #nm for 1200 g/mm from Model 207V Manual
MC_lamp= 'Xe' #Which lamp is on  Xe or D2?
MC_entrnace_slit_posiiton = 2 #mm 
# Traget_spectral_bandpass= 0.62 #nm 
Spectral_resolution=MC_dispersion_scale*MC_entrnace_slit_posiiton
print(f"Spectral Resoluton:{Spectral_resolution}")

# MC_exit_slit_posiiton=Traget_spectral_bandpass/MC_dispersion_scale #mm 
print(f"Entrance slit is at {np.round(MC_entrnace_slit_posiiton,2)} mm")
MC_entrnace_slit_posiiton=MC_entrnace_slit_posiiton #mm 
print(f"Exit slit is at {np.round(MC_entrnace_slit_posiiton,2)} mm. Set to same as enterance slit.")


Spectral Resoluton:2.48
Entrance slit is at 2 mm
Exit slit is at 2 mm. Set to same as enterance slit.


In [32]:
# # List of ExperimentSettings for PIXIS
# method_list = [method for method in dir(ExperimentSettings) if method.startswith('__') is False]
# print(method_list)

In [2]:
mcapi.whereishome()

631.26

In [3]:
#Port information for the controllers
MCPort = 'COM4'
#Shutter_Port= 'COM5'
#Filterwheel_Port= 'COM3'

In [5]:
Experiment_PIXIS_default='PIXIS_MC_Default'
# Experiment_PIXIS_default='DemoExp'
status, auto, experiment=pixis_load_experiment(Experiment_PIXIS_default)

if status==True: 
    print("PIXIS is ready")
    

Connection successful
PIXIS is ready


In [ ]:
# # pixis_get_value(ExperimentSettings.OnlineExportFormat)
# LF_folder=pixis_get_value(ExperimentSettings.OnlineExportOutputOptionsCustomDirectory)
# LF_folder

In [25]:
#intial experiment settings
#Export files as fits 
exp_filenames_basename='exp_Xe_5000ms_2_micron_slit_Filter_1'
exp_folder_name=exp_filenames_basename+'_180to700nm_65C'
# exp_bias_folder_name='bias'
# exp_bias_folder_name='darks'
parent_directory='C:\\Users\\User\\Documents\\LightField'
exp_filnames_basename_bias=exp_filenames_basename+'_bias'
exp_filnames_basename_dark=exp_filenames_basename+'_dark_'

exp_filenames_attachdate=False #to attach date to filename
exp_filenames_attachtime=False #to attach time to filename
exp_filenames_attachincrement=True #to attach automatic increment to filename

pixis_set_value(ExperimentSettings.OnlineExportEnabled,True) #enables export of acquired data other than the native *.spe format
pixis_set_value(ExperimentSettings.OnlineExportFormat,1) #  1 for FITS format
pixis_set_value(ExperimentSettings.OnlineExportOutputOptionsExportedFilesOnly,True)
pixis_set_value(ExperimentSettings.OnlineExportFormatOptionsIncludeExperimentInformation,True)
pixis_set_value(CameraSettings.ShutterTimingMode,2) # 2 open the shutter when exposure is taken, 3 for shutter remains closed when exposure is taken.  When Trigger ouput set to "Shutter"
exp_directory=pixis_set_folder(exp_folder_name,parent_directory)

pixis_set_value(ExperimentSettings.FileNameGenerationAttachDate,exp_filenames_attachdate)
pixis_set_value(ExperimentSettings.FileNameGenerationAttachTime,exp_filenames_attachtime)
pixis_set_value(ExperimentSettings.FileNameGenerationAttachIncrement,exp_filenames_attachincrement)
pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filenames_basename)

pixis_set_value(CameraSettings.AdcSpeed,2) #set to 2 MHz, other option is 0.1 MHz
pixis_set_value(CameraSettings.AdcGain,1) #set Adc Gain to 1, other values are 2, 3 

print(f" Detector Temperature = {pixis_get_current_temperature()}")


Created empty folder C:\Users\User\Documents\LightField\exp_Xe_8000ms_0_5_micron_slit_180to700nm_65C for storing experiment files
Custom directory set to C:\Users\User\Documents\LightField\exp_Xe_8000ms_0_5_micron_slit_180to700nm_65C 
Current Temperature:23.0
 Detector Temperature = 23.0


In [26]:
#WARNING: set detector temperature to below zero only when the system is in vacuuum and bring back the detector to ambient after experiment. 
#est -65
pixis_set_temperature(-65)

Current Status: UnLocked
Current Temperature:20.0
Detector temperature is being changed to -65. Current temperature is 20.0
Current Status: UnLocked
Current Temperature:18.0
Detector temperature is being changed to -65. Current temperature is 18.0
Current Status: UnLocked
Current Temperature:17.0
Detector temperature is being changed to -65. Current temperature is 17.0
Current Status: UnLocked
Current Temperature:15.0
Detector temperature is being changed to -65. Current temperature is 15.0
Current Status: UnLocked
Current Temperature:14.0
Detector temperature is being changed to -65. Current temperature is 14.0
Current Status: UnLocked
Current Temperature:12.0
Detector temperature is being changed to -65. Current temperature is 12.0
Current Status: UnLocked
Current Temperature:11.0
Detector temperature is being changed to -65. Current temperature is 11.0
Current Status: UnLocked
Current Temperature:10.0
Detector temperature is being changed to -65. Current temperature is 10.0
Current 

In [5]:
# Optional code to check ports connected to the computer (optional)
# Checking the ports connected to the computer
portlist=mcapi.getports()

COM1: USB Serial Port (COM1) [USB VID:PID=0403:6001 SER=A6040W3ZA]
COM3: USB Serial Port (COM3) [USB VID:PID=0403:6001 SER=A65892C1A]
COM4: USB Serial Port (COM4) [USB VID:PID=0403:6001 SER=A6040X2DA]
COM5: USB Serial Port (COM5) [USB VID:PID=0403:6001 SER=A65893GGA]


In [4]:
#setup monochromator 

#make sure the monochrmator COM ports are correct

#Optional code to check ports connected to the computer (optional)
#Checking the ports connected to the computer
# portlist=mcapi.getports()

#Port information for the controllers
MCPort = 'COM4' #com port for monochromator 
# Shutter_Port= 'COM5' #com port for shutter for RS232 shutter control. Not required when shutter operated with PIXIS BNC 
#Filterwheel_Port= 'COM3' #com port for filter wheel. Not used at the moment. 

#home the monochromator. # Do this for every time a new experiment is run 
mcapi.home(MCPort)
#mcapi.stop(MCPort)
#monochromator is all set for running the experiment.  

Limit Status Could Not Be Read. Error: could not open port 'COM4': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)


UnboundLocalError: local variable 'ser' referenced before assignment

In [31]:
import monochromatorapi as mcapi
MCPort = 'COM4'
#mcapi.stop(MCPort)
#mcapi.home(MCPort)
# print("MC at home")
start_wl=180.0 #nm
end_wl=700.0 #nm
wl_step=5.0 #nm 
exp_time=5000.0 #milleseconds 
#set number of dark frames per wavelength
ndark=int(1) #int
mcapi.go_to_fromhome(MCPort,start_wl)
#Routine: take 10 bias frames, then scan from start nm to end nm and take 1 dark and 1 exposure frame in steps of nm. 
#Taking Bias Frames 
n_bias=10
exp_filnames_basename_bias_pre=exp_filenames_basename+'_bias_pre'
pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filnames_basename_bias_pre)
pixis_take_bias_frames(n_bias)
pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filenames_basename)

current_wl=start_wl
while current_wl<=end_wl:
    print(f"Taking exposure at {np.round(current_wl,2)} nm")
    exp_filename=exp_filenames_basename+"_wl"+str(np.round(current_wl,2)).replace(".", "_")
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filename)
    pixis_take_exposure(exp_time)
    dark_filename=exp_filnames_basename_dark+"_wl"+str(np.round(current_wl,2)).replace(".", "_")
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,dark_filename)
    pixis_take_dark_frames(ndark,exp_time)
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filenames_basename)
    #input("Press any key continue")
    print(f"Going to {current_wl+wl_step} nm")
    mcapi.go_to_from(MCPort,current_wl,float(current_wl+wl_step))
    current_wl+=wl_step
mcapi.home(MCPort)
exp_filnames_basename_bias_post=exp_filenames_basename+'_bias_post'
pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filnames_basename_bias_post)
pixis_take_bias_frames(n_bias)
pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName,exp_filenames_basename)
print("Scan complete")
pixis_set_temperature(23)
pixis_get_current_temperature()

home circuit enabled, prepared to home
scanner is above home so moving down to home
decreasing wavelength at 23KHz
Limits status is:34
scan controller stopped
decreasing wavelength for 3 revolutions
increasing wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz this will take about 15 seconds
scan controller stopped
homing successful
disabled home circuit
scan controller is moving for -451.0nm
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
m

In [19]:
#close the automation sequence. Your data has been already saved. 
auto.Dispose()


In [1]:
pixis_set_temperature(23)

NameError: name 'pixis_set_temperature' is not defined